In [ ]:
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
import pandas as pd
import glob
import seaborn as sns

In [ ]:
reconfiles=sorted(glob.glob('./PredictedTiles/*'))
inputfiles=sorted(glob.glob('../TMA4-CellTiles/*'))
print(len(reconfiles))

In [ ]:

plt.rcParams['figure.figsize']=[20,80]
fig,ax=plt.subplots(8,2)

chan=0
n=0
for idx in range(0,8):
    reconimg=imread(reconfiles[idx])[:,:,chan]
    inputimg=imread(inputfiles[idx])[:,:,chan]
    ax[n,0].imshow(np.dstack((inputimg,inputimg,inputimg)))
    ax[n,1].imshow(np.dstack((reconimg,reconimg,reconimg)))
    ax[n,0].axis('off')
    ax[n,1].axis('off')
    n+=1
plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
import math


In [ ]:
###COMPUTE SSIM

In [ ]:
from skimage.metrics import structural_similarity as ssim

In [ ]:

SsimMat=[]
for idx in range(0,len(inputfiles))[::100]:
    SsimVec=[]
    reconimg=imread(reconfiles[idx])
    inputimg=imread(inputfiles[idx])
    for chan in range(0,25):
        SsimVec.append(ssim(inputimg[:,:,chan], reconimg[:,:,chan],data_range=inputimg[:,:,chan].max() - inputimg[:,:,chan].min()))
    SsimMat.append(SsimVec)
    if idx%10000==0:
        print(idx)
        
        

In [ ]:
out=np.reshape(SsimMat,(len(SsimMat),25))
out[out!=out]=1

In [ ]:
print('Mean: '+str(np.mean(np.mean(out,axis=0))))
print('Std: '+str(np.mean(np.std(out,axis=0))))

In [ ]:
reconfiles=sorted(glob.glob('./PredictedTiles/*'))
inputfiles=sorted(glob.glob('../TMA4-CellTiles/*'))

In [ ]:
import os
names = [os.path.basename(x) for x in reconfiles]
fileset=set(names)
len(fileset)

In [ ]:
num=1000
PredMat=[]
TrueMat=[]

for i,file in enumerate(names):
    
    name=file.split('/')[-1]
    predimage=imread('PredictedTiles/'+name)
    trueimage=imread('TMA4-CellTiles/'+name)
    
    predmask=np.sum(predimage,axis=2)>0
    predmeanVec=np.mean(predimage[predmask,:],axis=0)
    PredMat.append(predmeanVec)

    truemask=np.sum(trueimage,axis=2)>0
    truemeanVec=np.mean(predimage[truemask,:],axis=0)
    TrueMat.append(truemeanVec)
    
    if i%1000==0:
        print(i)

    #if i==num:
    #    break

PredMat=np.reshape(PredMat,(len(names) +1,25))
Preds=PredMat[1:,:]

TrueMat=np.reshape(TrueMat,(len(names) +1,25))
Trues=TrueMat[1:,:]




In [ ]:

CorrVec=[] #Corr for all stains
SelCorrVec=[] #Corr for Withheld Set Only
UsedChannels=np.array([0,21,5]) #Channels in Reduced Set

plt.rcParams['figure.figsize']=[5,5]
from scipy.stats import pearsonr
from scipy.stats import spearmanr
for ch in range(0,25):
    
    rho, pval = spearmanr(Preds[:,ch], Trues[:,ch])
    print('Spearman rank: '+str(rho))
    CorrVec.append(rho)

    if np.sum(UsedChannels==ch)==0:
        SelCorrVec.append(rho)
    
    plt.scatter(Pred[:,ch],DataTemp[:,ch],alpha=.01)
    plt.xlim(0,255)
    plt.ylim(0,255)
    plt.show()

In [ ]:
print('Full Stain Correlation')
print(np.mean(CorrVec))

print('Withheld Stain Correlation')
print(np.mean(SelCorrVec))

In [ ]:
print('Full Stain Variance')
print(np.var(CorrVec))

print('Withheld Stain Variance')
print(np.var(SelCorrVec))

In [ ]:
import phenograph
from scipy import stats

#Calculate Cluster for True Data
sc = StandardScaler(with_mean=True, with_std=True)
MeanMat=sc.fit_transform(Trues)

In [ ]:
embeddings= umap.UMAP(random_state=42).fit_transform(MeanMat[:,1:])

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(MeanMat[:,1:])
clusters=kmeans.labels_

In [ ]:
plt.rcParams['figure.figsize']=[10,10]
Well_colors={0:'red', 1:'palevioletred', 2:'darkorange', 3:'gold',4:'green',5:'yellowgreen',6:'blue',7:'darkturquoise',8:'indigo',9:'mediumpurple'}
plt.scatter(embeddings[:,0],embeddings[:,1],c=pd.DataFrame(clusters)[0].map(Well_colors))

In [ ]:
communities, graph, Q = phenograph.cluster(MeanMat[:,1:],k=500,min_cluster_size=1000,prune=True)
to_plot=np.hstack((np.reshape(communities,(len(communities),1)),embeddings))
outlier=to_plot[(to_plot[:,0] <0)]
to_plot=to_plot[(to_plot[:,0] >=0)]

In [ ]:
plt.rcParams['figure.figsize']=[10,10]
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

cs = get_cmap(np.max(to_plot[:,0]))
plt.scatter(outlier[:,1],outlier[:,2],c='grey',alpha=.2)
plt.scatter(to_plot[:,1],to_plot[:,2],c=to_plot[:,0],cmap=cs)

In [ ]:
#Calculate Clusters from Predictions

In [ ]:
encodings=sc.fit_transform(Preds)
embeddings= umap.UMAP(random_state=42).fit_transform(encodings)

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(encodings[:,:])
reducedclusters=kmeans.labels_
reducedclusters

In [ ]:
to_plot=pd.concat((pd.DataFrame(embeddings),pd.DataFrame(reducedclusters)),axis=1)
to_plot.columns=['X','Y','C']

In [ ]:
plt.rcParams['figure.figsize']=[10,10]

#Colors determined from percent overlap

#Full
#Well_colors={0:'red', 1:'palevioletred', 2:'darkorange', 3:'gold',4:'green',5:'yellowgreen',6:'blue',7:'darkturquoise',8:'indigo',9:'mediumpurple'}

#Corrbased
#Well_colors={3:'red', 7:'palevioletred', 5:'darkorange', 0:'gold', 8:'green', 1:'yellowgreen', 6:'blue', 4:'darkturquoise', 2:'indigo',9:'mediumpurple'}

#Gradbased
Well_colors={1:'red', 4:'palevioletred', 8:'darkorange', 0:'gold',7:'green',9:'yellowgreen',2:'blue',6:'darkturquoise',5:'indigo',3:'mediumpurple'}


#SubSpace
#Well_colors={9:'red', 6:'palevioletred', 8:'darkorange', 4:'gold',0:'green',2:'yellowgreen',5:'blue',3:'darkturquoise',1:'indigo',7:'mediumpurple'}

#Random
#Well_colors={2:'red', 9:'palevioletred', 0:'darkorange', 8:'gold',1:'green',5:'yellowgreen',3:'blue',7:'darkturquoise',6:'indigo',4:'mediumpurple'}
            
plt.scatter(to_plot['X'],to_plot['Y'],c=to_plot['C'].map(Well_colors))

In [ ]:
communities2, graph, Q = phenograph.cluster(encodings,k=500,min_cluster_size=1000,prune=True)

In [ ]:
to_plot2=np.hstack((np.reshape(communities2,(len(communities2),1)),embeddings))
outlier2=to_plot2[(to_plot2[:,0] <0)]
to_plot2=to_plot2[(to_plot2[:,0] >=0)]

In [ ]:
plt.rcParams['figure.figsize']=[10,10]
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

cs = get_cmap(np.max(to_plot2[:,0]))
plt.scatter(outlier2[:,1],outlier2[:,2],c='grey',alpha=.2)
plt.scatter(to_plot2[:,1],to_plot2[:,2],c=to_plot2[:,0],cmap=cs)

In [ ]:
#NMI KMeans

In [ ]:
from sklearn.metrics import normalized_mutual_info_score
normalized_mutual_info_score(clusters,reducedclusters)

In [ ]:
#NMI PhenoGraph

In [ ]:
Combined=np.hstack((np.reshape(communities,(len(communities),1)),(np.reshape(communities2,(len(communities2),1)))))
Combined=Combined[(Combined >=0).all(axis=1)] #remove all outliers

In [ ]:
normalized_mutual_info_score(Combined[:,0],Combined[:,1])